In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
import tensorflow as tf
from tensorflow import keras
import os
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.optimizers.legacy import Adam



/var/folders/5z/26yp_z91779dj66ktjgtwdph0000gn/T/ipykernel_15810/2283024198.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv("datasets/train.csv")

split = 0.8
df = df.sample(frac=1).dropna()
train = df[:int(0.8*df.shape[0])]
test = df[int(0.8*df.shape[0]):]

x_train, y_train = train["title"], train["label"]
x_test, y_test = test["title"], test["label"]

x_train_enc = [one_hot(words,10000) for words in x_train]
x_test_enc = [one_hot(words,10000) for words in x_test]

sent_length=10
x_train_enc_padded = pad_sequences(x_train_enc,padding='pre',maxlen=sent_length)

x_test_enc_padded = pad_sequences(x_test_enc,padding='pre',maxlen=sent_length)

In [3]:
x_train_enc_padded.shape

(14628, 10)

In [48]:
def create_model():
    model = tf.keras.Sequential([
    keras.layers.Dense(512, input_shape=(10, )),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation = "sigmoid"),
    ])
    model.compile(optimizer = Adam(lr=1e-3), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=[tf.keras.metrics.BinaryAccuracy()])
    return model

In [75]:
model = create_model()
model.summary()


Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 512)               5632      
                                                                 
 dropout_15 (Dropout)        (None, 512)               0         
                                                                 
 dense_31 (Dense)            (None, 1)                 513       
                                                                 
Total params: 6145 (24.00 KB)
Trainable params: 6145 (24.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [76]:
checkpoint_path = "training_1/checkpoints"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)


In [77]:
model.fit(x_train_enc_padded, y_train, epochs = 20, validation_data=(x_test_enc_padded, y_test), callbacks=[cp_callback])


Epoch 1/20


/Users/athulanoop/.local/share/mise/installs/python/3.10/lib/python3.10/site-packages/keras/src/backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


421/458 [==========================>...] - ETA: 0s - loss: 191.6139 - binary_accuracy: 0.6548
Epoch 1: saving model to training_1/checkpoints
458/458 [==============================] - 1s 1ms/step - loss: 185.6550 - binary_accuracy: 0.6542 - val_loss: 51.7566 - val_binary_accuracy: 0.7255
Epoch 2/20
416/458 [==========================>...] - ETA: 0s - loss: 85.6636 - binary_accuracy: 0.6484
Epoch 2: saving model to training_1/checkpoints
458/458 [==============================] - 1s 1ms/step - loss: 83.2463 - binary_accuracy: 0.6482 - val_loss: 28.9469 - val_binary_accuracy: 0.7017
Epoch 3/20
421/458 [==========================>...] - ETA: 0s - loss: 41.4392 - binary_accuracy: 0.6505
Epoch 3: saving model to training_1/checkpoints
458/458 [==============================] - 1s 1ms/step - loss: 40.0599 - binary_accuracy: 0.6514 - val_loss: 18.4452 - val_binary_accuracy: 0.7353
Epoch 4/20
429/458 [===========================>..] - ETA: 0s - loss: 17.1437 - binary_accuracy: 0.6524
Epoch 4:

In [78]:
loss, acc = model.evaluate(x_test_enc_padded,y_test, verbose=2)
print(f"model, accuracy: {100*acc:5.2f}%")

115/115 - 0s - loss: 0.6398 - binary_accuracy: 0.6689 - 84ms/epoch - 732us/step
model, accuracy: 66.89%


In [79]:
inp = ["FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart"]

input_proces = encode_input_data(inp)
out = model.predict(input_proces)
out

1/1 [==============================] - 0s 38ms/step


array([[0.89881784]], dtype=float32)

In [80]:
model_loaded = create_model()

In [81]:
loss_loaded, acc_loaded = model_loaded.evaluate(x_test_enc_padded, y_test)
print(f"Untrained model, accuracy: {100*acc_loaded:5.2f}%")


115/115 [==============================] - 0s 590us/step - loss: 228.9534 - binary_accuracy: 0.5885
Untrained model, accuracy: 58.85%


In [82]:
model_loaded.load_weights(checkpoint_path)

In [83]:
loss_loaded_trained, acc_loaded_trained = model_loaded.evaluate(x_test_enc_padded, y_test)
print(f"Trained model, accuracy: {100*acc_loaded_trained:5.2f}%")

115/115 [==============================] - 0s 572us/step - loss: 0.6398 - binary_accuracy: 0.6689
Trained model, accuracy: 66.89%


In [86]:
out = model_loaded.predict(input_proces)
out

1/1 [==============================] - 0s 34ms/step


array([[0.89881784]], dtype=float32)

In [71]:
modelServer = load_model(
        os.path.join("training_1/checkpoints")
    )


In [72]:
inp = ["FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart"]

input_proces = encode_input_data(inp)


In [73]:
out = model.predict(input_proces)
out

1/1 [==============================] - 0s 42ms/step


array([[0.15152659]], dtype=float32)